# **Part 3: Text Classification Testing**

In this section, comprehensive testing was done

Done By: Wong Wen Bing 230436M

In [1]:
# pip install openpyxl pandas 

In [2]:
import pandas as pd

import test dataset

In [3]:
testset=pd.read_excel('Data_test.xlsx')
testset.head()

,description,species
0,Looking for fun and interesting facts about a ...,Black-naped Oriole
1,Giant Panda. Grey Wolf. Canis lupus. Proboscis...,Black-naped Oriole
2,javanicus displayed during feeding such as wal...,Javan Myna
3,Black-naped Oriole: Explore more topics. Name;...,Black-naped Oriole
4,NaN,Javan Myna


clean and pre-process dataset

In [4]:
from nltk.corpus import stopwords #stopwords
from nltk.stem.wordnet import WordNetLemmatizer #lemmatization
import re
from nltk.tokenize import word_tokenize

def preprocess_text(text):
    lemma=WordNetLemmatizer() #lemmatization
    stopwordss=stopwords.words('english') #list of stopwords
    text = text.lower()  # Convert to lowercase
    # text = re.sub(r'\w*\d\w*', '', text)  # Remove words with numbers
    # text = re.sub(r'\W+', ' ', text)  # Remove special characters
    text = re.sub(r'[^a-z\s]', ' ', text)
    text = re.sub(r'\s\s+',' ', text)
    words=word_tokenize(text)
    text=[]
    for word in words: 
        if word not in stopwordss: 
            normalized = lemma.lemmatize(word)
            text.append(normalized)
    return ' '.join(text)

In [5]:
testset.drop_duplicates(inplace=True)
testset.dropna(inplace=True)
testset['cleaned_text'] = testset['description'].apply(preprocess_text)
testset.head()

,description,species,cleaned_text
0,Looking for fun and interesting facts about a ...,Black-naped Oriole,looking fun interesting fact black naped oriol...
1,Giant Panda. Grey Wolf. Canis lupus. Proboscis...,Black-naped Oriole,giant panda grey wolf canis lupus proboscis mo...
2,javanicus displayed during feeding such as wal...,Javan Myna,javanicus displayed feeding walking jumping ho...
3,Black-naped Oriole: Explore more topics. Name;...,Black-naped Oriole,black naped oriole explore topic name descript...
5,"The Javan myna, also known as the white-vented...",Javan Myna,javan myna also known white vented myna specie...


Import Model

In [6]:
import pickle
with open('Logistic Regression.pkl', 'rb') as f: 
    model=pickle.load(f)

C:\Users\wongw\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.5.1 when using version 1.3.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\wongw\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator LabelBinarizer from version 1.5.1 when using version 1.3.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\wongw\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator OneVsRestClassifier from v

In [7]:
model

OneVsRestClassifier(estimator=LogisticRegression(multi_class='deprecated'))

In [8]:
import pickle
with open('Count Vectorizer.pkl', 'rb') as f: 
    cv=pickle.load(f)

C:\Users\wongw\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator CountVectorizer from version 1.5.1 when using version 1.3.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [10]:
cv

CountVectorizer(ngram_range=(1, 2), stop_words='english')

In [11]:
y_test=testset['species']

In [13]:
import pickle
with open('MWE Tokenizer.pkl', 'rb') as f: 
    tk=pickle.load(f)

def using_mwe(x):
    txt=[]
    texts = tk.tokenize(x.split(' '))
    for text in texts: 
        txt.append(text)
    return ' '.join(txt)

# Use tokenize method 
testset['chunked']=testset['cleaned_text'].apply(using_mwe)

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
test=testset['chunked'].tolist()
vector=cv.transform(test)
y_testresults=model.predict(vector)

ValueError: X has 9705 features, but LogisticRegression is expecting 9684 features as input.

In [40]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test, y_testresults)
cm

array([[12,  0,  1,  3],
       [ 0,  6,  0,  0],
       [ 1,  0,  6,  3],
       [ 0,  0,  0,  7]])

In [42]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_testresults, target_names=['Black-naped Oriole','Collared Kingfisher','Javan Myna','Little Egret']))

                     precision    recall  f1-score   support

 Black-naped Oriole       0.92      0.75      0.83        16
Collared Kingfisher       1.00      1.00      1.00         6
         Javan Myna       0.86      0.60      0.71        10
       Little Egret       0.54      1.00      0.70         7

           accuracy                           0.79        39
          macro avg       0.83      0.84      0.81        39
       weighted avg       0.85      0.79      0.80        39



In [ ]:
testset['naive_bayes_pred']=(nb.predict(vector))[0]

In [ ]:
def accuracy(df): 
    # Calculate True Positives (TP): cases where prediction matches the actual species
    correct_predictions = df['species'] == df['naive_bayes_pred']
    tp = correct_predictions.sum()
    
    # Calculate total records
    total_records = len(df)
    
    # Calculate accuracy
    accuracy = tp / total_records if total_records > 0 else 0
    
    # Calculate False Positives (FP): cases where the prediction is wrong
    false_positives = (~correct_predictions).sum()
    
    return accuracy
    
accuracy(testset)

In [44]:
testset.head()

,description,species,cleaned_text,log_reg_pred
0,Looking for fun and interesting facts about a ...,Black-naped Oriole,looking fun interesting fact black naped oriol...,Black-naped Oriole
1,Giant Panda. Grey Wolf. Canis lupus. Proboscis...,Black-naped Oriole,giant panda grey wolf canis lupus proboscis mo...,Black-naped Oriole
2,javanicus displayed during feeding such as wal...,Javan Myna,javanicus displayed feeding walking jumping ho...,Black-naped Oriole
3,Black-naped Oriole: Explore more topics. Name;...,Black-naped Oriole,black naped oriole explore topic name descript...,Black-naped Oriole
5,"The Javan myna, also known as the white-vented...",Javan Myna,javan myna also known white vented myna specie...,Black-naped Oriole


In [52]:
testset.to_excel('newtest.xlsx')